# Append Data from SDE into Feature Service

The script utilizes both Esri's Destkop Python library 'arcpy' & the web Python API.  Connect to ArcGIS Online or Enterprise
parse through various datapoints and package them up for append operation.

In [3]:
# Import modules
import json
import arcpy
import getpass
import shutil
import time
from arcpy import env
from arcgis.gis import *

In [4]:
# Funtion to delete temporary files
def delete_json(temp_file, value):
    if value == 0:
        if os.path.exists(temp_file):
            try:
                os.remove(temp_file)
            except OSError as e:  ## if failed, report it back to the user ##
                print("Error: %s - %s." % (e.filename, e.strerror))
    else:
        try:
            if arcpy.Exists(temp_file):
                arcpy.Delete_management(temp_file)
            else:
                pass
        except (arcpy.ExecuteError, arcpy.ExecuteWarning) as e:
            print("Error: %s" % e)

In [5]:
# Yield successive n-sized chunks from l.
def chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i + n]

## Connect to ArcGIS Online - Python API
Estbalish connection, search for content by name to return the item's ID.  Once found, connect to relevant index dataset, and truncate the service in preparation for data load.

In [6]:
# Utilize getpass for hiding password for demonstration purposes
password = getpass.getpass()

········


In [7]:
# Establish connection to a WebGIS organization
# ArcGIS Online
#gis = GIS('http://www.arcgis.com', 'abrown_citygov', password)

# For ArcGIS Enterprise, connect as "https:<server DNS>/<web adaptor for portal>"
gis = GIS('http://neenterprise.esri.com/portal', 'username', password)  

In [8]:
# Search by title
search = gis.content.search(query="title: PhillyCrimeExample", item_type="Feature Service")
search

[<Item title:"PhillyCrimeExample" type:Feature Layer Collection owner:abrown_citygov>]

In [9]:
# From search list, return the index for service one is interseted in connecting to
item = search[0]
itemID = item.id
itemID

'1d1ee09e66084db0a189ee0e86ba9689'

In [10]:
# Establish connection to Feature Layer item
feature_layer_item = gis.content.get(itemID)
flayers = feature_layer_item.layers
flayer = flayers[0]
flayer

<FeatureLayer url:"https://services6.arcgis.com/0p6i4J6xhQas4Unf/arcgis/rest/services/PhillyCrimeExample/FeatureServer/0">

In [11]:
# Truncate the dataset, cannot be utlized on layers with synch
flayer.manager.truncate()

# Alternative Option, utilize delete_features method in class FeatureLayer
# flayer.delete_features(where="1=1")

{'success': True}

## Connect to Enterprise Geodatabase

Establish connection to enterprise geodatabase.  List feature classes, locate dataset of interest. Convert dataset into a json file. 

In [13]:
# Establish arcpy desktop workspace
wspace = r'D:\NYPD\complaints.gdb'
env.workspace = wspace

In [14]:
# List Feature Classes in Enterprise Geodatabase
fclist = arcpy.ListFeatureClasses()

In [15]:
# Locate dataset of interest
for item in fclist:
    if item == 'complaints':
        # Establish dataset name
        datasetname = item.rsplit('.', 1)[1]
        
        # Create temporary in memory layer for conversion to json
        arcpy.MakeFeatureLayer_management(item, "layer")
        
        # Create placeholder for temporary json file
        temp_json = r'D:\Scratch\jsons_' + datasetname + '.json'
        
        # Make sure temporary json does not exist
        delete_json(temp_json, 0)

        # Convert dataset into json file
        arcpy.FeaturesToJSON_conversion("layer", temp_json)

        # Delete temporary memory layer
        arcpy.Delete_management("layer")
        
    else:
        pass

## Prepare data for insert into Feature Service

Convert to a file geodatabase (only method to utilize append and preserve schema matching without a data dictionary). Zip up the file geodatabase and add the content to ArcGIS Online. Append the new data to an existing feature service.  Delete all temporary files.

In [16]:
# Open json exported from enterprise dataset
file = open(temp_json)
content = json.loads(file.read())

# Identify json sections to build into smaller json dictionary in subsequent sections.
display = (content["displayFieldName"])
alias = (content["fieldAliases"])
geomtype = (content["geometryType"])
spatref = (content["spatialReference"])
fields = (content["fields"])
new_features = (content["features"])

print('Total Number of features: %s' % (len(new_features)))

# Utilize the chunks generator to create groups of 50000 features
# Upper limit seems to be in the neighborhood of 250K - 350K, depending on the complexity of the datasets.
new_items = chunks(new_features, 50000)

# Loop through items created into smaller chunks
for i, item in enumerate(new_items):
    
    # Create a new diciontary to build proper json (reconstructing with less features)
    json_dict = {}
    json_dict['displayFieldName'] = display
    json_dict['fieldAliases'] = alias
    json_dict['geometryType'] = geomtype
    json_dict['spatialReference'] = spatref
    json_dict['fields'] = fields
    json_dict['features'] = item

    # Output variables for temporary output type & locations
    output = r'D:\Scratch\jsons\new_json_' + str(i) + '.json'
    outzipdir = r'D:\Scratch\zip'
    outshapedir = r'D:\Scratch\fgdb'
    output2 = r'D:\Scratch\fgdb\item' + str(i) + '.gdb'
    fname = output2.rsplit('\\', 1)[-1]

    # Create a File Geodatabase to be zipped
    print(os.path.join(outshapedir, fname))
    arcpy.CreateFileGDB_management(outshapedir, fname)
    zipname = fname.rsplit('.', 1)[0]
   
    # Open temporary subset json file, write reconstructed dictionary as content.
    with open(output, 'w') as outfile:
        json.dump(json_dict, outfile, ensure_ascii=False)

    # Convert new subset json file to feature class in the file geodatabase
    arcpy.JSONToFeatures_conversion(in_json_file=output, out_features=os.path.join(output2, str(fname).split('.')[0]))

    # Delete subset json temporary file
    delete_json(output, 0)
    finalname = outzipdir + '\\' + zipname + '.gdb'
    
    # Establish name of the output zip file
    deletezip = finalname + '.zip'

    # Zip up the contents of the fgdb folder properly to ensure data hierarchy 
    shutil.make_archive(finalname, "zip", outshapedir)
    time.sleep(30)
    delete_json(output2, 1)

    # Create item properties 
    item_properties = {'title': fname,
                       'description': 'Append testing',
                       'type': 'File Geodatabase',
                       'tags': 'test',
                       'thumbnail': r"C:\Users\alex8694\Pictures\Logos\esri.png" }
    
    print('Adding Item...')
    
    # Add content to ArcGIS Online/Enterprise as an item
    gis.content.add(item_properties=item_properties, data=deletezip)

    print('Done Adding Item')

    # Establish query based upon item index name
    query = "title:" + str(fname)
    print(query)
    
    # Extra time to ensure upload process completed 
    time.sleep(10)
    
    # Search for new temporary file geodatabase item
    search = gis.content.search(query=query, item_type="File Geodatabase")
    print(search)
    item = search[0]
    
    # Temporary item id
    new_item = item.id
    
    # Complete the append operation on the base service
    try:
        flayer.append(item_id=new_item, upload_format='filegdb')
    except TypeError:
        # Numerous errors occur when item uploaded has not full loaded
        
        # Repeat time to wait and try again
        time.sleep(10)

        # Search for new temporary file geodatabase item
        search = gis.content.search(query=query, item_type="File Geodatabase")
        item = search[0]

        # Temporary item id
        new_item = item.id
        flayer.append(item_id=new_item, upload_format='filegdb')

    # Delete temporary file geodatabase on ArcGIS Online/Enterprise
    item_for_deletion = gis.content.get(new_item)
    item_for_deletion.delete()

    # Delete temporary zip file
    delete_json(deletezip, 0)

Total Number of features: 104265
D:\Scratch\fgdb\item0.gdb
Adding Item...
Done Adding Item
title:item0.gdb
[<Item title:"item0.gdb" type:File Geodatabase owner:abrown_citygov>]
D:\Scratch\fgdb\item1.gdb
Adding Item...
Done Adding Item
title:item1.gdb
[<Item title:"item1.gdb" type:File Geodatabase owner:abrown_citygov>]
D:\Scratch\fgdb\item2.gdb
Adding Item...
Done Adding Item
title:item2.gdb
[<Item title:"item2.gdb" type:File Geodatabase owner:abrown_citygov>]


In [17]:
# Close master json file
file.close()

# Return total record count
query_result = flayer.query(where='1=1', out_fields='*')
print(len(query_result))

# Delete master json file for dataset
delete_json(temp_json, 0)

104265
